# 집값 예측하기 (Predicing House Prices)

In [ ]:
!ls -al ../data/

## Import Graphlab

graphlab을 import합니다. (에러가 나는 경우 'install graphlab' 노트북을 열어서 설치해야 합니다.)

In [ ]:
import graphlab

## Read some product review data

SFrame 데이터를 읽어 와서 **sales**라는 변수에 저장합니다. graphlab에서 제공하는 SFrame이라는 메소드를 사용해야 합니다.
'../data/home_data.gl/'이라는 폴더를 지정해야 합니다.

    graphlab.SFrame(폴더위치)

In [ ]:
sales = graphlab.SFrame('../data/home_data.gl/')

## sales 내용 살펴 보기
sales라고 입력하면 sales의 내용을 확인할 수 있습니다. **sales**라고 입력 후 실행해 보세요.

In [ ]:
sales

어떤 데이터들이 들어 있나요?

# 데이터 들여다 보기

우리의 목적은 주어진 데이터를 이용하여 집값을 예측하는 모델을 만드는 것입니다. 먼저 sqft_living (실평수)가 집값과 직접적인 관계가 있는지 확인해 보겠습니다.

이를 위해서 x축을 'sqft_living'으로, y축을 'price'로 하는 Scatter plot으로 그려봅시다.

먼저 아래 명령어를 실행해서 그래프가 이 노트북에 출력되도록 합시다.

In [ ]:
graphlab.canvas.set_target('ipynb')


In [ ]:
sales.show(view="Scatter Plot", x='sqft_living', y='price')

다음 줄에 아래 구문을 입력하고 x값에 'sqft_living'을, y값에 'price'를 지정합시다.

    sales.show(view="Scatter Plot", x=x값, y=y값)

이 그래프는 어떤 의미일까요?

# 실평수로 집값을 예측하는 단순한 회귀 모델 만들기
자, sqft_living과 price간에 상관관계가 있다는 걸 확인했기 때문에 sqft_living을 가지고 집값을 예측하는 가장 단순한 선형 회귀 모델을 만들어 보겠습니다.

전체 데이터인 Sales를 80:20의 비율로 training set과 test set을 나누고자 합니다. 여기서 SFrame.random_split 메소드는 SFrame 데이터를 랜덤하게 지정된 비율에 따라 나누어 줍니다.

    SFrame.random_split(1번데이터가차지하는비율, seed=시드값)

실무에서는 seed값을 랜덤하게 사용하겠지만 여기서는 모두가 동일한 결과를 얻기 위해서 시드값을 0으로 지정합니다.

sales 데이터를 **train_data**와 **test_data**라는 변수에 나눠서 저장해야 합니다. 이를 위해 아래 명령어를 실행합시다.

In [ ]:
train_data, test_data = sales.random_split(.8, seed=0)

sales 데이터의 타입과 길이를 확인해 봅시다.

In [ ]:
print type(sales), " | ", len(sales)

동일한 방법으로 **train_data**와 **test_data**의 타입과 길이를 확인해 봅시다. 아래 2개의 빈칸에 각각 입력해 보세요.

In [ ]:
print type(train_data), " | ", len(train_data)

In [ ]:
print type(test_data), " | ", len(test_data)

2개 모두 SFrame이라는 타입이 나와야 합니다. train_data와 test_data의 길이는 각각 17384, 4229가 나와야 합니다.

## sqft_living을 단일 feature로 하는 회귀 모델 만들기

회귀 모델을 생성하는 메소드는 다음과 같습니다.

    graphlab.linear_regression.create(학습데이터, target=예측하고자하는값의컬럼이름, features=feature로사용하고자하는데이터의컬럼이름리스트, validataion_set=None)

주의: features에는 반드시 **리스트**를 지정해야 합니다.

주의: 예측하고자 하는 값의 컬럼 이름, feature로 사용하고자 하는 데이터의 컬럼 이름 등은 학습데이터 안에 반드시 존재해야 합니다.

이렇게 생성한 모델을 **sqft_model**이라는 곳에 저장해야 합니다. 

In [ ]:
sqft_model = graphlab.linear_regression.create(train_data, target='price', features=['sqft_living'], validation_set=None)


# 단순 모델 평가하기
모델이 성공적으로 생성되었나요?

모델을 검증하기 전에 먼저 test_data에 속한 집들의 평균 가격을 확인합니다.

test_data의 집값 (price) 컬럼을 먼저 확인해 봅시다. 

In [ ]:
test_data['price']

이제는 print 함수를 이용해서 출력해 봅시다.

In [ ]:
print test_data['price']

그럼 이 값들의 평균을 구해 봅시다. SArray.mean()이라는 메소드를 사용하면 해당 컬럼의 평균을 구할 수 있습니다.

In [ ]:
print test_data['price'].mean()


sqft_living의 평균도 확인해 봅시다.

In [ ]:
print test_data['sqft_living'].mean()

### RMSE로 모델 검증하기

train_data를 이용해서 학습시킨 모델에 test_data를 입력하여 집값을 예측한 결과를 검증해 봅시다. 이를 위해 graphlab은 evaluate라는 메소드를 제공하고 그 결과로 max_error와 RMSE이라는 척도를 제공합니다.

    model.evaluate(데이터)
    
print 명령을 이용해서 결과값을 출력해 봅시다.

In [ ]:
sqft_model.evaluate(test_data)

RMSE가 약 \$255,191인 것을 확인할 수 있습니다. **RMSE (Root Mean Square Error)**는 앞으로 자주 사용하게 될 척도입니다.

# 방금 만든 모델에 test_data를 입력해서 예측한 값을 그래프에 그려보기

Matplotlib는 파이썬 시각화 라이브러리입니다. 우리는 matplotlib를 사용하여 그래프를 그리고자 합니다.

먼저 아래 명령을 실행하세요.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### 데이터셋 그래프 상에 표현하기

matplot의 plot메소드를 사용하면 그래프에 **점**을 찍을 수 있습니다.

    plt.plot(x값, y값, 마커)

먼저 우리가 현재 보유한 test_data의 값을 시각화 해 봅시다. 그러기 위해 test_data의 x값 (sqft_living)과 y값 (price)을 '.'마커로 찍어봅시다. 

In [ ]:
plt.plot(test_data['sqft_living'], test_data['price'], '.')

그 다음으로 test_data의 x값과 그 값에 해당하는 모델의 예측값을 '-'마커로 찍어봅시다. 모델의 예측값은 아래와 같이 predict 메소드를 이용해서 구할 수 있습니다.

    model.predict(데이터)

In [ ]:
plt.plot(test_data['sqft_living'], sqft_model.predict(test_data), '-')

In [ ]:
plt.plot(test_data['sqft_living'], sqft_model.predict(test_data), '.')

### 관측값과 예측값 비교하기

이제 2개의 그래프를 한번에 같이 그려 봅시다. 2개 이상의 데이터셋을 같이 화면에 그리고자 할 때는 아래와 같이 이어서 작성하면 됩니다.

    plt.plot(x1값, y1값, 마커1, x2값, y2값, 마커2, ...)

In [ ]:
plt.plot(test_data['sqft_living'], sqft_model.predict(test_data), '-', test_data['sqft_living'], test_data['price'], '.')

파란색 점은 관측값을 의미하며 오렌지색 선 (점들의 연결)은 sqft_living을 feature로 사용한 sqft_model의 예측값에 해당합니다. 

** 실제 관측값과 우리가 예측한 값을 비교해 볼 수 있습니다.**

위에서 확인한 price와 sqft_living 평균값들이 그래프에서 어디쯤에 있는지 확인해 봅시다.

### 모델의 coefficients

모델이 학습한 coefficients를 확인하기 위해서는 get 메소드를 사용합니다.

    model.get('coefficients')

In [ ]:
sqft_model.get('coefficients')

이번 학습을 통해 y = wx + b라는 linear equation을 이용한 회귀 모델의 파라미터 (w, b)가 위와 같이 정해졌습니다. 여기서 intercept는 bias를 의미합니다.

# 데이터의 다른 커럼들을 살펴 보자

위에서는 sqft_living 데이터 하나만을 가지고 모델을 만들었습니다. 이제 여기서는 더 나은 모델을 만들기 위해서 다른 features를 살펴 봅시다.

다음의 컬럼이름들을 요소로 하는 **my_features**라는 리스트를 만드세요.

    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode'

In [ ]:
my_features =['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']

### Filtering

SFrame[컬럼이름리스트]를 하면 SFrame에 있는 데이터 중에 리스트에 이름이 있는 컬럼들만 선택됩니다. 이를 SFrame의 **필터링**이라고 부릅니다.

In [ ]:
sales[my_features]

### SFrame.show

SFrame.show 메소드를 사용하면 더 많은 정보를 확인할 수 있습니다. 

    SFrame.show()

In [ ]:
sales[my_features].show()

show메소드의 다른 옵션을 이용하면 BoxWhisker Plot을 그릴 수 있습니다.

    sales.show(view='BoxWhisker Plot', x=x값, y=y값)
    
이제 x값에 **'zipcode'**를, y값에 **'price'**를 지정해 봅시다.

In [ ]:
sales.show(view='BoxWhisker Plot', x='zipcode', y='price')

그래프 하단의 파란색 경계를 클릭해서 드래그하면 보고자 하는 데이터의 범위를 변경할 수 있습니다. 최대한 넓게 영역을 만들어서 가장 '비싼' 집들이 많은 zipcode를 찾아 봅시다. 어느 zip code인가요?

# 더 많은 피쳐들을 이용해서 회귀 모델 만들기

회귀 모델을 생성하는 메소드는 다음과 같습니다.

    graphlab.linear_regression.create(학습데이터, target=예측하고자하는값의관측값의컬럼이름, features=사용하고자하는feature데이터의컬럼이름리스트, validataion_set=None)

이번에는 features에 my_features를 지정합니다. 그리고 이렇게 생성한 모델을 **my_features_model**이라는 변수에 저장합시다. 

In [ ]:
my_features_model = graphlab.linear_regression.create(train_data, target='price', features=my_features, validation_set=None)


한번 더 확인을 위해 my_features를 print 함수를 이용해서 출력해 봅시다.

In [ ]:
print my_features

## 더 많은 피쳐로 만들 모델과 단순 모델의 결과 비교하기

sqft_model과 my_features_model를 이용해서 test_data의 예측값이 잘 나오는지 검증해 봅시다. 앞에서 했듯이 evaluate 메소드를 이용합시다.

    model.evaluate(data)

In [ ]:
my_features_model.evaluate(test_data)

두 모델의 RMSE를 비교해 봅시다. 

my_features_model을 사용했더니 RMSE가 \$255,191에서 \$179,542로 낮아진 것을 확인할 수 있습니다.

두 모델 중 어느 모델이 더 잘 맞춘다고 할 수 있나요?

# 학습한 모델을 이용해서 3개의 집값을 예측해 보자

먼저 첫번째로 알아볼 집은 시애틀의 *평균적인* 집에 해당합니다.

id가 5309101200인 집을 선택해서 **house1**이라는 변수에 저장해 봅시다.

In [ ]:
house1 = sales[sales['id']=='5309101200']

In [ ]:
house1

house1을 출력해서 데이터를 확인합시다.

비교를 위해 house1의 집가격만 출력해 봅시다.

In [ ]:
print house1['price']

sqft_model을 이용해서 house2의 집값을 예측해 봅시다. 이전과 마찬가지로 predict 메소드를 이용하면 됩니다.

    model.predict(data)

In [ ]:
print sqft_model.predict(house1)

이번에는 my_features_model을 이용하여 house1의 집값을 예측해 봅시다.

In [ ]:
print my_features_model.predict(house1)

이 경우에는 더 많은 피쳐를 사용한 my_features_model이 하나의 피쳐를 사용한 sqft_model보다 더 부정확한 예측을 하는 것을 확인할 수 있습니다. 

하지만 평균적으로는 my_features_model이 더 나은 예측을 할 겁니다. (왜 그럴까요?)

## 조금 더 팬시한 집의 가격을 예측해 봅시다.

id가 1925069082인 집을 선택해서 **house2**라는 변수에 저장해 봅시다.

In [ ]:
house2 = sales[sales['id']=='1925069082']

house2를 출력해서 데이터를 확인합시다.

In [ ]:
house2

<img src="https://ssl.cdn-redfin.com/photo/1/bigphoto/302/734302_0.jpg">

sqft_model을 이용해서 house2의 집값을 예측해 봅시다. 이전과 마찬가지로 predict 메소드를 이용하면 됩니다.

predict(data)

In [ ]:
print house2['price']

In [ ]:
print sqft_model.predict(house2)

이번에는 my_features_model을 이용하여 house2의 집값을 예측해 봅시다.

In [ ]:
print my_features_model.predict(house2)

이번 경우에는 더 많은 피쳐를 사용한 모델 (my_features_model)이 더 좋은 예측을 했습니다. 이 집이 물가에 있는 집이라는 점, 그리고 더 많은 장점을 가진 집이라는 점 등을 고려하면 이는 당연한 일입니다. 

다시 한번 my_features를 출력해 봅시다.

In [ ]:
print my_features

두 집 (house1과 house2)의 my_features에 해당하는 값들을 비교해 봅시다.

## 마지막, 슈퍼-팬시한 집을 살펴 봅시다.

마지막 집은 유명한 시애틀 사람의 아주 거대한 집입니다. 바로 빌 게이츠의 집입니다. 이 집의 데이터는 sales에 없기 때문에 새로이 만들어 봅시다.

In [ ]:
bill_gates = {'bedrooms':[8], 
              'bathrooms':[25], 
              'sqft_living':[50000], 
              'sqft_lot':[225000],
              'floors':[4], 
              'zipcode':['98039'], 
              'condition':[10], 
              'grade':[10],
              'waterfront':[1],
              'view':[4],
              'sqft_above':[37500],
              'sqft_basement':[12500],
              'yr_built':[1994],
              'yr_renovated':[2010],
              'lat':[47.627606],
              'long':[-122.242054],
              'sqft_living15':[5000],
              'sqft_lot15':[40000]}

먼저 type함수를 이용해서 bill_gates의 유형을 확인해 봅시다.

In [ ]:
print type(bill_gates)

우리가 사용하는 값은 SFrame인데 bill_gates 변수는 dictionary인 것을 확인할 수 있습니다. 따라서 먼저 dictionary 데이터를 사용하기 전에 아래 함수를 이용해서 SFrame으로 변환합시다.

    graphlab.SFrame(dict)

이렇게 변환된 값을 **house3**에 저장합시다.

** 주의: Jupyter에서 하나의 변수 이름을 재활용하는 것은 가급적이면 피해야 합니다. bill_gates라는 변수명을 다시 쓰지 마세요. **

In [ ]:
house3 = graphlab.SFrame(bill_gates)

my_features_model을 이용해서 이 집의 가격을 예측해 봅시다.

In [ ]:
print my_features_model.predict(house3)

모델은 이 집이 $13M 이상의 가격일 거라고 예측했습니다. 하지만 실제 가격은 아마 훨씬 더 비쌀겁니다. (sales 데이터 안에 이 정도로 팬시한 고가의 집 데이터들이 거의 없기 때문에 이 모델이 제대로 값을 예측하기는 어려울 것입니다.)

# Week2 Quiz

## Quiz1

** Selection and Summary Statistics **

** 문제: 평균 집값이 가장 높은 zipcode 지역의 평균 집값은 얼마인가요? **

1. zipcode가 98039인 집들을 골라낸다.
2. 해당 집들의 가격 컬럼만을 선택한다.
3. mean 메소드를 이용해서 집값을 구한다.
4. int함수를 이용해서 정수화할 수 있다.

In [ ]:
house01 = sales[sales['zipcode']=='98039']

In [ ]:
house01['price'].mean()

In [ ]:
print int(house01['price'].mean())

## Quiz2

** Flitering Data ** 

** 문제: sqft_living이 2000 이상 4000 이하인 집들이 차지하는 비율은? **

- https://turi.com/products/create/docs/generated/graphlab.SFrame.html 


1. 위 링크를 보고 & 오퍼레이터에 대해 이해할 것 
2. len 함수를 쓰면 개수를 셀 수 있습니다.
3. 비율을 어떻게 구해야 할까요?
4. float 함수를 사용하면 소수점 이하 연산이 가능합니다.

In [ ]:
house02 = sales[(sales['sqft_living'] >= 2000) & (sales['sqft_living'] <= 4000)]

In [ ]:
len(house02), len(sales)

In [ ]:
print float(len(house02))/float(len(sales))

## Quiz 3

** advaced_features를 이용해서 새로운 모델을 만들어 보고 이 모델을 이용해서 test_data의 rmse를 측정해 보자. **

** 문제: 이 모델과 my_features_model과의 rmse 차이는 몇 $인가? **

In [ ]:
advanced_features = [
'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode',
'condition', # condition of house				
'grade', # measure of quality of construction				
'waterfront', # waterfront property				
'view', # type of view				
'sqft_above', # square feet above ground				
'sqft_basement', # square feet in basement				
'yr_built', # the year built				
'yr_renovated', # the year renovated				
'lat', 'long', # the lat-long of the parcel				
'sqft_living15', # average sq.ft. of 15 nearest neighbors 				
'sqft_lot15', # average lot size of 15 nearest neighbors 
]

In [ ]:
adv_f_m = graphlab.linear_regression.create(train_data,target='price',features=advanced_features,validation_set=None) 

In [ ]:
afm=adv_f_m.evaluate(test_data)['rmse']
mfm=my_features_model.evaluate(test_data)['rmse']
print afm
print mfm
print abs(afm-mfm)